In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import plotnine

from plotnine import *

# get param set
paramsets =  pd.read_csv('../../output_random_rows.csv')
paramsets = paramsets.drop(paramsets.columns.tolist()[-1],axis=1)
outputvalsname =  ['wat','cs']
column_names = outputvalsname + paramsets.columns.tolist()
paramsets.drop('theta',inplace=True, axis=1)

df = pd.read_pickle("./pset99_allcell.pkl")  # cid, pSet, rr, 1 or 9


nKonz = 9
volIdx=nKonz+1
lenIdx=volIdx+1
stIdx=lenIdx+1
orgIdId=stIdx+1
relLenId=orgIdId+1
coordId = relLenId+1
segIdxid = coordId+1

nToGet = coordId#elements + 3d coordinates of y , node+vol+len+st 


In [81]:
df.shape

(17, 99)

In [2]:
konzcs = df.loc[1]# pSet, rr, 9


vols = df.loc[volIdx]# pSet, rr, 9
lens = df.loc[lenIdx]# pSet, rr, 1
orgIds = df.loc[orgIdId]# pSet, rr, 1
reLens =  df.loc[relLenId]
coords =  df.loc[coordId]
segIdx =  df.loc[segIdxid]
sts = df.loc[stIdx]

In [3]:
dfnew = pd.DataFrame(columns=['locCylId','coords','vols','lens','reLens','pSet'])

for pSet in range(99):
    
    cyltr = len(orgIds.loc[pSet])# 21 or 22
    locCylId = np.array([np.full(9,ii) for ii in range(cyltr)])
    locCellId = np.full((cyltr,9),np.array([i for i in range(9)]))
    
    myData = pd.DataFrame({'locCylId':locCylId.reshape(-1)})
    
    myData['locCellId']= locCellId.reshape(-1)
    myData['coords']= coords.loc[pSet].reshape(-1)
    
    myData['wat']=  df.loc[0].loc[pSet].reshape(-1)
    myData['cs']=  df.loc[1].loc[pSet].reshape(-1)
    myData['cl']=  df.loc[2].loc[pSet].reshape(-1)
    myData['coa']=  df.loc[3].loc[pSet].reshape(-1)
    myData['cod']=  df.loc[4].loc[pSet].reshape(-1)
    myData['cca']=  df.loc[5].loc[pSet].reshape(-1)
    myData['ccd']=  df.loc[6].loc[pSet].reshape(-1)
    myData['css2']= df.loc[7].loc[pSet].reshape(-1)
    myData['co2']= df.loc[8].loc[pSet].reshape(-1)
    
    myData['vols']= vols.loc[pSet].reshape(-1)
    
    lens__ = lens.loc[pSet]
    lens_ = np.array([np.full(9, ll) for ll in lens__])
    reLens__ = reLens.loc[pSet]
    reLens_ = np.array([np.full(9, ll) for ll in reLens__])
    myData['lens']= lens_.reshape(-1)
    myData['reLens']= reLens_.reshape(-1)
    
    sts__ = sts.loc[pSet]
    sts_ = np.array([np.full(9, ll) for ll in sts__])
    segIdx__ = segIdx.loc[pSet]
    segIdx_ = np.array([np.full(9, ll) for ll in segIdx__])
    myData['sts']= sts_.reshape(-1)
    myData['segIdx']= segIdx_.reshape(-1)
    
    myData['pSet'] = pSet
    
    dfnew = pd.concat([dfnew, myData], ignore_index=True)

In [4]:
dfbu = dfnew

set inner and outer radious from root center

In [5]:
dfnew['rIn'] = -1
dfnew['rOut'] = -1

dfnew.loc[((dfnew['sts']==0))&((dfnew['locCellId']==0)),'rIn']=0.05
dfnew.loc[((dfnew['sts']==3))&((dfnew['locCellId']==0)),'rIn']=0.05
dfnew.loc[((dfnew['sts']==1))&((dfnew['locCellId']==0)),'rIn']=0.02
dfnew.loc[((dfnew['sts']==2))&((dfnew['locCellId']==0)),'rIn']=0.02

dfnew.loc[((dfnew['locCellId']==0)),'rOut'] = (dfnew.loc[((dfnew['locCellId']==0)),'coords']
                                               - dfnew.loc[((dfnew['locCellId']==0)),'rIn'])*2+dfnew.loc[((dfnew['locCellId']==0)),'rIn']


for cellid in range(1,9):
    dfnew.loc[((dfnew['locCellId']==cellid)),'rIn'] =np.array(list(dfnew.loc[((dfnew['locCellId']==cellid-1)),'rOut']))
    dfnew.loc[((dfnew['locCellId']==cellid)),'rOut'] = (dfnew.loc[((dfnew['locCellId']==cellid)),'coords']
                                               - dfnew.loc[((dfnew['locCellId']==cellid)),'rIn'])*2+dfnew.loc[((dfnew['locCellId']==cellid)),'rIn']

In [6]:
dfnew['rIn2'] = -1
dfnew['rOut2'] = -1
dfnew['coords2'] =np.nan

dfnew.loc[((dfnew['locCellId']==0)),'rIn2']=0.

dfnew.loc[((dfnew['locCellId']==0)),'rOut2'] = (dfnew.loc[((dfnew['locCellId']==0)),'coords']
                                               - dfnew.loc[((dfnew['locCellId']==0)),'rIn'])*2+dfnew.loc[((dfnew['locCellId']==0)),'rIn2']

dfnew.loc[((dfnew['locCellId']==0)),'coords2']=(dfnew.loc[((dfnew['locCellId']==0)),'rIn2'] +
                                                           dfnew.loc[((dfnew['locCellId']==0)),'rOut2'])/2

for cellid in range(1,9):
    dfnew.loc[((dfnew['locCellId']==cellid)),'rIn2'] =np.array(list(dfnew.loc[((dfnew['locCellId']==cellid-1)),'rOut2']))
    dfnew.loc[((dfnew['locCellId']==cellid)),'rOut2'] = (dfnew.loc[((dfnew['locCellId']==cellid)),'coords']
                                               - dfnew.loc[((dfnew['locCellId']==cellid)),'rIn'])*2+dfnew.loc[((dfnew['locCellId']==cellid)),'rIn2']
    dfnew.loc[((dfnew['locCellId']==cellid)),'coords2'] = (dfnew.loc[((dfnew['locCellId']==cellid)),'rIn2'] +
                                                           dfnew.loc[((dfnew['locCellId']==cellid)),'rOut2'])/2

In [9]:
dfnew.head()#n=30)

,locCylId,coords,konzcs,vols,lens,reLens,pSet,locCellId,wat,cs,...,ccd,css2,co2,sts,segIdx,rIn,rOut,rIn2,rOut2,coords2
0,0,0.054990,NaN,0.003448,1.0,1.0,0,0.0,0.000744,3.586009e-07,...,0.000009,0.000008,0.000004,0.0,0.0,0.050000,0.059980,0.000000,0.009980,0.004990
1,0,0.065966,NaN,0.004962,1.0,1.0,0,1.0,0.001071,3.585574e-07,...,0.000009,0.000008,0.000004,0.0,0.0,0.059980,0.071952,0.009980,0.021952,0.015966
2,0,0.079133,NaN,0.007141,1.0,1.0,0,2.0,0.001541,3.584819e-07,...,0.000009,0.000008,0.000004,0.0,0.0,0.071952,0.086314,0.021952,0.036314,0.029133
3,0,0.094928,NaN,0.010276,1.0,1.0,0,3.0,0.002218,3.583793e-07,...,0.000009,0.000008,0.000004,0.0,0.0,0.086314,0.103542,0.036314,0.053542,0.044928
4,0,0.113876,NaN,0.014788,1.0,1.0,0,4.0,0.003191,3.582561e-07,...,0.000009,0.000008,0.000004,0.0,0.0,0.103542,0.124210,0.053542,0.074210,0.063876


In [7]:
print(max(dfnew.loc[((dfnew['locCellId']==0)),'rIn']), min(dfnew.loc[((dfnew['locCellId']==8)),'rOut']))
print(max(dfnew.loc[((dfnew['locCellId']==0)),'rIn2']), 
      min(dfnew.loc[((dfnew['locCellId']==8)),'rOut2']), 
      max(dfnew.loc[((dfnew['locCellId']==8)),'rOut2']))
print(max(dfnew.loc[((dfnew['locCellId']==0)),'coords2']), 
      min(dfnew.loc[((dfnew['locCellId']==8)),'coords2']), 
      max(dfnew.loc[((dfnew['locCellId']==8)),'coords2']))
# perirhizal always bigger than theoreticall rhizosphere (3mm)

0.05 0.1032863052134135
0.0 0.08328630521341351 0.5445439630212967
0.00773921154585816 0.07467491248310894 0.45702422259471265


set inner and outer radious from root surface

keep perio zone

save data

In [8]:
dfnew.to_pickle('./dfperrhizal_allcell.pkl')

In [ ]:
# redefine vale in rhizosphere

In [9]:
dfnew =  pd.read_pickle("./dfperrhizal_allcell.pkl") 


In [ ]:
if False:
    # Sort the group by face_start to ensure correct order
    group = group.sort_values(by='rIn2')

    total_volume = np.sum(group['vols'] )

    # Calculate the total solute content in the entire domain
    total_solute_content = np.sum(group[namekonz]* group['vols'])

    # Determine if r1 is within the domain or beyond its outer boundary
    outer_face_end = group['rOut2'].max()
    totvol = 0
    if r1 >= outer_face_end:
        # r1 is beyond the outer boundary, so use the total solute content
        solute_content = total_solute_content
        partial_volume = total_volume
    else:
        # r1 is within the domain, calculate solute content up to r1
        mask = group['rOut2'] <= r1
        if np.any(mask):
            relevant_volumes = group['vols'][mask]
            relevant_concentrations = group[namekonz][mask]
            solute_content = np.sum(relevant_concentrations * relevant_volumes)
            partial_volume = np.sum(relevant_volumes)
            # Calculate contribution of the partially included cell at r1
            partial_start = group.loc[group['rOut2'] > r1, 'rIn'].values[0]
            partial_start_diff = group.loc[group['rOut2'] > r1, 'rIn'].values[0]-group.loc[group['rOut2'] > r1, 'rIn2'].values[0]
            partial_length = group.loc[group['rOut2'] > r1, 'lens'].values[0]
            partial_volume_ = ((r1+partial_start_diff)**2 - partial_start**2) * partial_length * np.pi
            partial_volume += partial_volume_

            partial_concentration = np.interp((r1-partial_start)/2, group['coords2'], group[namekonz])
            solute_content += partial_concentration * partial_volume_
        else:
            solute_content = 0.0

In [10]:
# Function to calculate the solute content for each domain up to r1
def calculate_solute_content(group, r1, namekonz):
    # Sort the group by face_start to ensure correct order
    group = group.sort_values(by='rIn2')
    
    total_volume = np.sum(group['vols'] )
    
    # Calculate the total solute content in the entire domain
    total_solute_content = np.sum(group[namekonz]* group['vols'])
    
    # Determine if r1 is within the domain or beyond its outer boundary
    outer_face_end = group['rOut2'].max()
    totvol = 0
    if r1 >= outer_face_end:
        # r1 is beyond the outer boundary, so use the total solute content
        solute_content = total_solute_content
        partial_volume = total_volume
    else:
        # r1 is within the domain, calculate solute content up to r1
        mask = group['rOut2'] <= r1
        if np.any(mask):
            relevant_volumes = group['vols'][mask]
            relevant_concentrations = group[namekonz][mask]
            solute_content = np.sum(relevant_concentrations * relevant_volumes)
            partial_volume = np.sum(relevant_volumes)
            # Calculate contribution of the partially included cell at r1
            partial_start = group.loc[group['rOut2'] > r1, 'rIn'].values[0]
            partial_start2 = group.loc[group['rOut2'] > r1, 'rIn2'].values[0]
            # for vol computation  see ditance needed to add to go from rel to actual radius
            partial_start_diff = group.loc[group['rOut2'] > r1, 'rIn'].values[0]-group.loc[group['rOut2'] > r1, 'rIn2'].values[0]
            partial_length = group.loc[group['rOut2'] > r1, 'lens'].values[0]
            partial_volume_ = ((r1+partial_start_diff)**2 - partial_start**2) * partial_length * np.pi
            partial_volume += partial_volume_
            
            partial_concentration = np.interp((r1+partial_start2)/2, group['coords2'], group[namekonz])
            solute_content += partial_concentration * partial_volume_
        else:
            solute_content = 0.0
    
    return pd.Series({
        #'cylId': group['locCylId'].iloc[0],
        'partialQ_'+namekonz: solute_content,
        'partial_volume': partial_volume,
        'st': group['sts'].iloc[0],
        #'pSet': group['pSet'].iloc[0]
    })


In [11]:
#get everything per unit of osil
dfnew['theta']=dfnew['wat']/dfnew['vols']
dfnew['cssoil']=dfnew['cs']*dfnew['theta']
dfnew['clsoil']=dfnew['cl']*dfnew['theta']
# Create a list to store results for different concentration ranges
results = []
obsDis = 0.3
for namekonz_ in [ 'theta','cssoil','clsoil','coa','cod','cca','ccd','css2','co2']:
    print('namekonz_', namekonz_)
    result_df = dfnew.groupby(['locCylId','pSet']).apply(calculate_solute_content, r1=obsDis, namekonz=namekonz_).reset_index()
    results.append(result_df)

# Concatenate the results into a single DataFrame
final_result = pd.concat(results)

namekonz_ theta
namekonz_ cssoil
namekonz_ clsoil
namekonz_ coa
namekonz_ cod
namekonz_ cca
namekonz_ ccd
namekonz_ css2
namekonz_ co2


In [12]:
from functools import reduce
# Columns to merge on
merge_columns = ['locCylId',  'pSet','st', 'partial_volume']

# Merge the DataFrames
merged_df = reduce(lambda left, right: pd.merge(left, right, on=merge_columns), results)

In [13]:
merged_df.to_pickle('./allcellupto3mm.pkl')

In [71]:
results4mm = []
obsDis = 0.4
for namekonz_ in [ 'theta','cssoil','clsoil','coa','cod','cca','ccd','css2','co2']:
    print('namekonz_', namekonz_)
    result_df = dfnew.groupby(['locCylId','pSet']).apply(calculate_solute_content, r1=obsDis, namekonz=namekonz_).reset_index()
    results4mm.append(result_df)

# Concatenate the results into a single DataFrame
#final_result = pd.concat(results)

namekonz_ theta
namekonz_ cssoil
namekonz_ clsoil
namekonz_ coa
namekonz_ cod
namekonz_ cca
namekonz_ ccd
namekonz_ css2


KeyboardInterrupt: 

In [67]:
final_result = results[0]

In [69]:
merged_df4mm = reduce(lambda left, right: pd.merge(left, right, on=merge_columns), results4mm)

,locCylId,pSet,partialQ_theta,partial_volume,st,partialQ_cssoil,partialQ_clsoil,partialQ_coa,partialQ_cod,partialQ_cca,partialQ_ccd,partialQ_css2,partialQ_co2
0,0,0,0.043164,0.200000,0.0,5.998799e-10,1.524960e-06,3.119199e-10,1.854668e-06,7.421398e-09,1.854936e-06,0.000002,8.408621e-07
1,0,1,0.043166,0.200000,0.0,4.885644e-09,1.633127e-06,1.178649e-07,1.742715e-06,4.701059e-08,1.826157e-06,0.000013,2.784769e-07
2,0,2,0.036237,0.166667,0.0,1.074736e-09,7.834426e-07,3.867846e-09,1.539474e-06,6.580774e-09,1.540964e-06,0.000004,2.543901e-07
3,0,3,0.043132,0.200000,0.0,3.140896e-09,1.218252e-06,1.826677e-07,1.676849e-06,8.154267e-08,1.867059e-06,0.000009,3.210757e-06
4,0,4,0.043088,0.200000,0.0,1.229272e-08,1.349293e-06,3.725793e-07,1.607809e-06,4.602473e-07,1.637071e-06,0.000033,3.959292e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45856,546,94,0.071197,0.320442,1.0,2.391082e-08,6.663581e-06,1.040253e-08,2.957595e-06,3.547098e-07,2.768813e-06,0.000021,2.451830e-06
45857,547,85,0.068783,0.321530,2.0,5.373358e-08,5.043134e-06,2.916498e-07,2.700474e-06,2.456439e-07,2.915783e-06,0.000057,1.508983e-06
45858,547,94,0.012793,0.059763,2.0,7.772471e-10,1.347833e-07,2.090924e-08,5.335921e-07,8.910079e-08,5.008186e-07,0.000006,5.544475e-07
45859,548,85,0.053783,0.250652,2.0,3.190254e-08,2.920939e-06,2.287976e-07,2.103725e-06,2.016465e-07,2.265616e-06,0.000046,1.195044e-06


In [35]:
dfset388 = dfset[dfset['locCylId']==int(dfset.loc[dfset['rOut2']==max(dfset['rOut2']),'locCylId'])]

In [42]:
dfset388.to_pickle('./dfset388.pkl')

In [38]:
dfset388.to_csv('your.csv', index=False)

In [32]:
dfset[dfset['rOut2']==max(dfset['rOut2'])]#388

,locCylId,coords,konzcs,vols,lens,reLens,pSet,locCellId,wat,cs,...,ccd,css2,co2,sts,segIdx,rIn,rOut,rIn2,rOut2,coords2
7451,388,0.477024,NaN,0.524634,1.0,1.0,1,8.0,0.116614,0.000003,...,0.000009,0.000067,0.000001,1.0,388.0,0.389504,0.564544,0.369504,0.544544,0.457024


In [29]:
print(np.sum(dfset0['vols']),np.sum(dfset0['vols']*dfset0['cs']) )

0.20000000000000007 5.830128987076197e-07


In [21]:
dfset

,locCylId,coords,konzcs,vols,lens,reLens,pSet,locCellId,wat,cs,...,ccd,css2,co2,sts,segIdx,rIn,rOut,rIn2,rOut2,coords2
3951,0,0.054990,NaN,0.003448,1.0,1.0,1,0.0,0.000744,0.000003,...,0.000009,0.000067,0.000001,0.0,0.0,0.050000,0.059980,0.000000,0.009980,0.004990
3952,0,0.065966,NaN,0.004962,1.0,1.0,1,1.0,0.001071,0.000003,...,0.000009,0.000067,0.000001,0.0,0.0,0.059980,0.071952,0.009980,0.021952,0.015966
3953,0,0.079133,NaN,0.007141,1.0,1.0,1,2.0,0.001541,0.000003,...,0.000009,0.000067,0.000001,0.0,0.0,0.071952,0.086314,0.021952,0.036314,0.029133
3954,0,0.094928,NaN,0.010276,1.0,1.0,1,3.0,0.002218,0.000003,...,0.000009,0.000067,0.000001,0.0,0.0,0.086314,0.103542,0.036314,0.053542,0.044928
3955,0,0.113876,NaN,0.014788,1.0,1.0,1,4.0,0.003192,0.000003,...,0.000009,0.000067,0.000001,0.0,0.0,0.103542,0.124210,0.053542,0.074210,0.063876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7834,431,0.090620,NaN,0.017010,1.0,1.0,1,4.0,0.003739,0.000004,...,0.000009,0.000079,0.000002,1.0,431.0,0.075683,0.105557,0.055683,0.085557,0.070620
7835,431,0.126391,NaN,0.033089,1.0,1.0,1,5.0,0.007274,0.000003,...,0.000009,0.000078,0.000002,1.0,431.0,0.105557,0.147224,0.085557,0.127224,0.106391
7836,431,0.176282,NaN,0.064368,1.0,1.0,1,6.0,0.014150,0.000003,...,0.000009,0.000078,0.000002,1.0,431.0,0.147224,0.205339,0.127224,0.185339,0.156282
7837,431,0.245866,NaN,0.125215,1.0,1.0,1,7.0,0.027525,0.000003,...,0.000009,0.000078,0.000002,1.0,431.0,0.205339,0.286394,0.185339,0.266394,0.225866


In [ ]:
obsDis = 0.3

dfnew = pd.DataFrame(columns=['locCylId','cs','reLens','pSet'])

for pSet in range(99):
    if len(coords.loc[pSet]) > 0:
        dfset = dfnew.loc[dfnew['pSet']==pSet]
        result_df = dfset.groupby('locCylId').apply(calculate_solute_content, r1=obsDis).reset_index()
    
        cyltr = len(orgIds.loc[pSet])
        locCylId = np.array([ii for ii in range(cyltr)])

        myData = pd.DataFrame({'locCylId':locCylId.reshape(-1)})

        #myData['locCellId']= locCellId#.reshape(-1)
        #myData['coords']= coords.loc[pSet]#.reshape(-1)

        #myData['konzcs']= konzcs.loc[pSet]#.reshape(-1)
        #myData['vols']= vols.loc[pSet]#.reshape(-1)

        #myData['lens']= lens.loc[pSet]
        myData['reLens']= reLens[pSet]
        myData['sts']= sts.loc[pSet]
        myData['segIdx']= segIdx.loc[pSet]
        myData['pSet'] =  pSet

        y_interp =np.array([np.interp(obsDis, coords.loc[pSet][cylloc],
                                      konzcs[pSet][cylloc]
                                     ) for cylloc in range(cyltr)])
        myData['konzc'] = y_interp
        dfnew = pd.concat([dfnew, myData], ignore_index=True)

BIN

In [94]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Sample input DataFrame with different cell centers between domains
data = {
    'domain': [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2],
    'center': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85],
    'concentration': [1.0, 1.2, 1.1, 1.3, 1.4, 1.5, 1.2, 1.3, 1.4, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8]
}

df = pd.DataFrame(data)

In [95]:

centers = group['center'].values
concentrations = group['concentration'].values

NameError: name 'group' is not defined

In [96]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Sample input DataFrame with different cell centers between domains
data = {
    'domain': [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2],
    'center': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85],
    'concentration': [1.0, 1.2, 1.1, 1.3, 1.4, 1.5, 1.2, 1.3, 1.4, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8]
}

df = pd.DataFrame(data)

# Function to perform interpolation and integration for a single domain
def interpolate_and_integrate(group, r1):
    centers = group['center'].values
    concentrations = group['concentration'].values
    
    # Sort centers and concentrations in case they are not sorted
    sorted_indices = np.argsort(centers)
    centers = centers[sorted_indices]
    concentrations = concentrations[sorted_indices]
    
    # Interpolation function
    interp_func = interp1d(centers, concentrations, kind='linear', fill_value="extrapolate")
    
    # Interpolated concentration at position r1
    concentration_at_r1 = interp_func(r1)
    
    # Calculate solute content up to r1 using the trapezoidal rule
    mask = centers <= r1
    if np.sum(mask) > 0:
        relevant_centers = np.append(centers[mask], r1)
        relevant_concentrations = np.append(concentrations[mask], concentration_at_r1)
        solute_content = np.trapz(relevant_concentrations, relevant_centers)
    else:
        solute_content = 0.0
    
    return pd.Series({
        'concentration_at_r1': concentration_at_r1,
        'solute_content_up_to_r1': solute_content
    })

# Define the input position r1
r1 = 0.5

# Apply the function to each group and combine the results
result_df = df.groupby('domain').apply(interpolate_and_integrate, r1=r1).reset_index()

print(result_df)

   domain concentration_at_r1  solute_content_up_to_r1
0       1                 1.4                  0.48000
1       2  2.4499999999999997                  1.00125


BIN

In [17]:

for pSet in range(99):
    cyltr = len(orgIds.loc[pSet])
    for cylid in range(cyltr):
        #lenCyl = dfnew[dfnew['pSet']==pSet][dfnew['locCylId']==cylid]['rIn'] 
        for cellid in range(1,9):
            rowcell = dfnew.loc[(dfnew['pSet']==pSet)&(
                dfnew['locCylId']==cylid)&(
                dfnew['locCellId']==cellid)].index.tolist()[0]
            lencyl = dfnew.at[rowcell,'lens']
            vol =  dfnew.at[rowcell,'vols']
            dfnew.at[rowcell,'rOut'] = np.sqrt(vol/(np.pi*lencyl) +dfnew.at[rowcell,'rIn']**2 )
            
            if cellid < 8:
                rownextcell = dfnew.loc[(dfnew['pSet']==pSet)&(dfnew['locCylId']==cylid)&(dfnew['locCellId']==0)].index.tolist()[0]
                dfnew.at[rownextcell,'rIn'] = dfnew.at[rowcell,'rOut']

,locCylId,coords,konzcs,vols,lens,reLens,pSet,locCellId,sts,segIdx
0,0,0.054990,3.586009e-07,0.003448,1.0,1.0,0,0.0,0.0,0.0
1,0,0.065966,3.585574e-07,0.004962,1.0,1.0,0,1.0,0.0,0.0
2,0,0.079133,3.584819e-07,0.007141,1.0,1.0,0,2.0,0.0,0.0
3,0,0.094928,3.583793e-07,0.010276,1.0,1.0,0,3.0,0.0,0.0
4,0,0.113876,3.582561e-07,0.014788,1.0,1.0,0,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
412744,483,0.090620,5.610470e-07,0.017010,1.0,1.0,98,4.0,1.0,483.0
412745,483,0.126391,5.610544e-07,0.033089,1.0,1.0,98,5.0,1.0,483.0
412746,483,0.176282,5.610581e-07,0.064368,1.0,1.0,98,6.0,1.0,483.0
412747,483,0.245866,5.610601e-07,0.125215,1.0,1.0,98,7.0,1.0,483.0


In [39]:
obsDis = 0.06

dfnew = pd.DataFrame(columns=['locCylId','cs','reLens','pSet'])

for pSet in range(99):
    if len(coords.loc[pSet]) > 0:
    
        cyltr = sum(orgIds.loc[pSet]==1)# 19-22
        locCylId = np.array([ii for ii in range(cyltr)])

        myData = pd.DataFrame({'locCylId':locCylId.reshape(-1)})

        #myData['locCellId']= locCellId#.reshape(-1)
        #myData['coords']= coords.loc[pSet]#.reshape(-1)

        #myData['konzcs']= konzcs.loc[pSet]#.reshape(-1)
        #myData['vols']= vols.loc[pSet]#.reshape(-1)

        #myData['lens']= lens.loc[pSet]
        myData['reLens']= reLens[pSet][orgIds.loc[pSet]==1]
        myData['pSet'] =  pSet

        y_interp =np.array([np.interp(obsDis, coords.loc[pSet][cylloc],
                                      konzcs[pSet][cylloc]
                                     ) for cylloc in range(cyltr)])
        myData['konzc'] = y_interp
        dfnew = pd.concat([dfnew, myData], ignore_index=True)

In [40]:
dfnew

,locCylId,cs,reLens,pSet,konzc
0,0,NaN,1.0,0,9.185186e-06
1,1,NaN,2.0,0,1.960936e-06
2,2,NaN,3.0,0,2.179401e-06
3,3,NaN,4.0,0,2.135895e-06
4,4,NaN,5.0,0,2.129707e-06
...,...,...,...,...,...
1992,15,NaN,16.0,98,6.567472e-07
1993,16,NaN,17.0,98,8.728945e-07
1994,17,NaN,18.0,98,9.250490e-07
1995,18,NaN,19.0,98,1.501308e-05


In [24]:
dfnew.head(n=10)#[dfnew['coords'] < 0.05]

,locCylId,coords,konzcs,vols,lens,reLens,pSet,locCellId,rIn,rOut
0,0,0.054990,1.073521e-05,0.003448,1.0,1.0,0,0.0,1.006989,-1.000000
1,0,0.065966,7.339324e-06,0.004962,1.0,1.0,0,1.0,-1.000000,1.000789
2,0,0.079133,4.557653e-06,0.007141,1.0,1.0,0,2.0,-1.000000,1.001136
3,0,0.094928,2.556139e-06,0.010276,1.0,1.0,0,3.0,-1.000000,1.001634
4,0,0.113876,1.326372e-06,0.014788,1.0,1.0,0,4.0,-1.000000,1.002351
5,0,0.136606,7.028279e-07,0.021280,1.0,1.0,0,5.0,-1.000000,1.003381
6,0,0.163873,4.505152e-07,0.030623,1.0,1.0,0,6.0,-1.000000,1.004862
7,0,0.196582,3.717496e-07,0.044068,1.0,1.0,0,7.0,-1.000000,1.006989
8,0,0.235820,3.537835e-07,0.063415,1.0,1.0,0,8.0,-1.000000,1.010042
9,1,0.022022,8.987728e-06,0.000560,1.0,2.0,0,0.0,1.001173,-1.000000


In [72]:
dfnew.loc[dfnew['locCellId']==0,'coords']# = 0.05

0        0.054990
9        0.022022
18       0.022174
27       0.022305
36       0.022305
           ...   
18675    0.023947
18684    0.023947
18693    0.057473
18702    0.022816
18711    0.023422
Name: coords, Length: 2080, dtype: float64

In [74]:
dfnew.to_pickle('./cyl10to11_celss2.pkl')

In [77]:
print(dfnew.loc[dfnew['locCellId']==0,'coords'].min(),dfnew.loc[dfnew['locCellId']==0,'coords'].max())
print(dfnew.loc[dfnew['locCellId']==8,'coords'].min(),dfnew.loc[dfnew['locCellId']==8,'coords'].max())

0.02202224498672597 0.057472616323468714
0.09610504338489201 0.47702422259471233


In [ ]:
dist = 0.0

In [ ]:
dfnew.loc[dfnew['locCellId']==0,'rIn']

In [ ]:
plt.scatter(dfnew['yX'],df['yZ'] , #np.log(df['cc']),np.log(df['co']), 
            c= np.log(df['cs']),#s=df['cs'],
            alpha=0.3, edgecolors ='none')
plt.rcParams.update({'figure.figsize':(5,5), 'figure.dpi':100})
#plt.title()
plt.colorbar()
plt.xlabel('cs')
plt.ylabel('co +cc')
plt.show()

In [41]:

cyltr = sum(orgIds.loc[pSet]==1)# 21
locCylId = np.array([np.full(9,ii) for ii in range(cyltr)])
locCellId = np.full((21,cyltr),np.array([i for i in range(cyltr)]))

myData = pd.DataFrame({'locCylId':locCylId.reshape(-1)})

#myData['locCellId']= locCellId.reshape(-1)

In [49]:
locCylId.shape

(22, 9)

In [50]:
locCellId.shape

(21, 9)

In [33]:
lens__ = lens.loc[pSet][orgIds.loc[0]==1]

In [36]:
lens_ = np.array([np.full(9, ll) for ll in lens__])

In [37]:
lens_

array([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [21]:
cc.shape

(21, 9)

In [26]:
cc[0]

array([0.05499005, 0.06596616, 0.07913313, 0.09492826, 0.11387611,
       0.136606  , 0.16387281, 0.19658214, 0.2358203 ])

In [25]:
cc.reshape(-1)

array([0.05499005, 0.06596616, 0.07913313, 0.09492826, 0.11387611,
       0.136606  , 0.16387281, 0.19658214, 0.2358203 , 0.02202224,
       0.02647568, 0.03182971, 0.03826646, 0.04600488, 0.05530819,
       0.06649286, 0.07993935, 0.09610504, 0.02217415, 0.02699516,
       0.03286432, 0.04000953, 0.04870822, 0.05929814, 0.07219047,
       0.08788579, 0.10699352, 0.02230469, 0.02744521, 0.03377047,
       0.04155351, 0.05113028, 0.0629142 , 0.07741395, 0.09525543,
       0.11720881, 0.02230469, 0.02744521, 0.03377047, 0.04155351,
       0.05113028, 0.0629142 , 0.07741395, 0.09525543, 0.11720881,
       0.02202224, 0.02647568, 0.03182971, 0.03826646, 0.04600488,
       0.05530819, 0.06649286, 0.07993935, 0.09610504, 0.0227442 ,
       0.02898565, 0.03693989, 0.04707692, 0.05999575, 0.07645977,
       0.09744183, 0.12418179, 0.15825972, 0.02363763, 0.03223614,
       0.04396248, 0.05995442, 0.08176365, 0.11150628, 0.15206819,
       0.20738504, 0.28282415, 0.0227442 , 0.02898565, 0.03693

In [ ]:
dfnew = pd.DataFrame(columns=['locCylId','coords','vols','lens','reLens','pSet'])

for pSet in range(99):
    
    cyltr = len(orgIds.loc[pSet])# 21 or 22
    locCylId = np.array([np.full(9,ii) for ii in range(cyltr)])
    locCellId = np.full((cyltr,9),np.array([i for i in range(9)]))
    
    myData = pd.DataFrame({'locCylId':locCylId.reshape(-1)})
    
    myData['locCellId']= locCellId.reshape(-1)
    myData['coords']= coords.loc[pSet].reshape(-1)
    
    myData['wat']=  df.loc[0].loc[pSet].reshape(-1)
    myData['cs']=  df.loc[1].loc[pSet].reshape(-1)
    myData['cl']=  df.loc[2].loc[pSet].reshape(-1)
    myData['coa']=  df.loc[3].loc[pSet].reshape(-1)
    myData['cod']=  df.loc[4].loc[pSet].reshape(-1)
    myData['cca']=  df.loc[5].loc[pSet].reshape(-1)
    myData['ccd']=  df.loc[6].loc[pSet].reshape(-1)
    myData['css2']= df.loc[7].loc[pSet].reshape(-1)
    myData['co2']= df.loc[8].loc[pSet].reshape(-1)
    
    myData['vols']= vols.loc[pSet].reshape(-1)
    
    lens__ = lens.loc[pSet]
    lens_ = np.array([np.full(9, ll) for ll in lens__])
    reLens__ = reLens.loc[pSet]
    reLens_ = np.array([np.full(9, ll) for ll in reLens__])
    myData['lens']= lens_.reshape(-1)
    myData['reLens']= reLens_.reshape(-1)
    
    sts__ = sts.loc[pSet]
    sts_ = np.array([np.full(9, ll) for ll in sts__])
    segIdx__ = segIdx.loc[pSet]
    segIdx_ = np.array([np.full(9, ll) for ll in segIdx__])
    myData['sts']= sts_.reshape(-1)
    myData['segIdx']= segIdx_.reshape(-1)
    
    myData['pSet'] = pSet
    
    dfnew = pd.concat([dfnew, myData], ignore_index=True)